In [4]:
%pip install azure-storage-blob azure-identity azure-ai-documentintelligence azure.ai.formrecognizer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.9/63.9 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.1/300.1 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.7/151.7 kB 4.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [12]:
from azure.core.credentials import AzureKeyCredential
from azure.ai.formrecognizer import DocumentAnalysisClient
from azure.storage.blob import BlobServiceClient, BlobClient
import os

output_directory = "./txt/"

def doing(file_name):

    # Azure Document Intelligence  서비스의 엔드포인트와 키, 모델 이름
    form_recognizer_endpoint = "https://globalaibootcamp-s1-01.cognitiveservices.azure.com/" # #메모했던 내용으로 수정
    form_recognizer_key = "75b7320edf304e52981f453cfda924ad" # #메모했던 내용으로 수정
    model_id = "first_model" #메모했던 내용으로 수정

    # Azure Blob Storage 계정 정보
    blob_account_name = "gabs101" #메모했던 내용으로 수정
    blob_account_key = "BP3TFJ+rMuxxCq/zDutEjqVCuuAfjijzEaUEe07wwsofI2XTpZ/+w0hRxMEV2wQYYIlCDvV7Ubez+AStoDXWeA==" # #메모했던 내용으로 수정

    # Blob Storage 클라이언트 생성
    blob_service_client = BlobServiceClient(
        account_url=f"https://{blob_account_name}.blob.core.windows.net",
        credential=blob_account_key
    )

    # Blob의 일반 링크
    # #blob에서 예제 1의 url : https://계정이름.blob.core.windows.net/gabs101/예제1.pdf 이렇게 되어 있는데 https://계정이름.blob.core.windows.net/gabs101/ 이걸 붙여넣기
    blob_url = f'https://gabs101.blob.core.windows.net/gabs101/{file_name}' # #메모했던 예제1  URL 일부 내용으로 수정. /{file_name}은 그대로 둬 주세요

    # Blob 다운로드
    blob_client = BlobClient.from_blob_url(blob_url, credential=blob_account_key)
    blob_data = blob_client.download_blob().readall()

    # Document Analysis 클라이언트 생성
    document_analysis_client = DocumentAnalysisClient(
        endpoint=form_recognizer_endpoint, credential=AzureKeyCredential(form_recognizer_key)
    )



    # Make sure your document's type is included in the list of document types the custom model can analyze
    poller = document_analysis_client.begin_analyze_document(model_id, blob_data)
    result = poller.result()

    txt_file_name = os.path.join(output_directory, f"{os.path.splitext(file_name)[0]}.txt")

    for document in result.documents:
        with open(txt_file_name, "w") as file:
            for name, field in document.fields.items():
                field_value = field.value if field.value else field.content
                print("{}:'{}'".format(name, field_value))
                file.write(f"{name}: '{field_value}'\n")

   

In [13]:
doing("예제1.pdf")

reason:'걍 참 가 했 음'
learnt:'Thank u so much 여 러 분'
information:'{'name': DocumentField(value_type=dictionary, value={'ROW1': DocumentField(value_type=string, value='개 구 리', content=개구리, bounding_regions=[BoundingRegion(page_number=1, polygon=[Point(x=1.7, y=5.905), Point(x=2.31, y=5.905), Point(x=2.31, y=6.155), Point(x=1.7, y=6.155)])], spans=[DocumentSpan(offset=136, length=3)], confidence=None)}, content=None, bounding_regions=[], spans=[], confidence=None), 'birth': DocumentField(value_type=dictionary, value={'ROW1': DocumentField(value_type=string, value='2023/03/23', content=2023/03/23, bounding_regions=[BoundingRegion(page_number=1, polygon=[Point(x=3.51, y=5.92), Point(x=4.655, y=5.92), Point(x=4.655, y=6.17), Point(x=3.51, y=6.17)])], spans=[DocumentSpan(offset=140, length=10)], confidence=None)}, content=None, bounding_regions=[], spans=[], confidence=None), 'phone': DocumentField(value_type=dictionary, value={'ROW1': DocumentField(value_type=string, value='010-2555-8999', con

In [60]:
import re

def extract(file_name):
    # 파일 경로 설정
    txt_file_path = "./txt/{file_name}"

    # 정보 추출을 위한 정규식 패턴
    information_pattern = r"information:'({.*?})'"
    satis_survey_pattern = r"satis_survey:'\[(.*?)\]'"

    # 파일 내용 읽기
    with open(txt_file_path, "r") as file:
        memo = file.read()

    #reason, learnt 줄은 그대로 유지. 건너띄기. information줄에선 information:만 남기고 뒤에 내용 아래 수행 정보로 대체

    # 정보 추출
    information_data = {}
    satis_survey_data = []

    # 정보 추출
    information_match = re.search(information_pattern, memo, re.DOTALL)
    if information_match:
        information_section = information_match.group(1)
        for match in re.finditer(r"'(\w+)': DocumentField.*?value='(.*?)'", information_section):
            key = match.group(1)
            value = match.group(2)
            information_data[key] = value

    # 만약 satis_survey가 존재하면 추출
    satis_survey_match = re.search(satis_survey_pattern, memo, re.DOTALL)
    if satis_survey_match:
        satis_survey_section = satis_survey_match.group(1)
        for match in re.finditer(r"'satis':\s*DocumentField.*?value='(.*?)'", satis_survey_section):
            satis_survey_data.append({'satis': match.group(1)})

    # 파일 내용 수정
    with open(txt_file_path, "w") as file:
        file.write(f"information: {information_data}\n")
        file.write(f"satis_survey: {satis_survey_data}\n")

    print("텍스트 파일이 성공적으로 수정되었습니다.")


텍스트 파일이 성공적으로 수정되었습니다.
